Lets Create a question answer section using T5 or other model on eli5 dataset.

If we use smaller llm models they can give the answer of the question but failed to give answer in human manner.
I have pre tained the bert model but it will only provide the extraceted asnwer. By extracted it mean the exact words will be generated againt the question.

Lets make it more fun. I think to use FIAS sementic serach for extraction relavent documents ffrom the model and then provide the text to the model for proper human answer.

In [86]:
from datasets import load_dataset
from transformers import T5TokenizerFast
import pandas as pd

In [87]:
dataset = load_dataset("eli5")
dataset

DatasetDict({
    train_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 272634
    })
    validation_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 9812
    })
    test_eli5: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 24512
    })
    train_asks: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 131778
    })
    validation_asks: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 2281
    })
    test_asks: Dataset({
       

lets combine the dataset as we only need to answer the question on all the possible topic we can have

WE are not going to train our model on this dataset. Insted we need to use this information for our sementic search.


In [88]:
dataset_titles = list (dataset.keys() )
dataset_titles

['train_eli5',
 'validation_eli5',
 'test_eli5',
 'train_asks',
 'validation_asks',
 'test_asks',
 'train_askh',
 'validation_askh',
 'test_askh']

In [89]:
dataset['test_asks'][1]

{'q_id': '3ko13p',
 'title': 'In the latest Avengers movie, there is a "Nexus" hub which routes every packet of traffic on the internet. What is the closest thing we have to a core of the internet in real life?',
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['cuz3k5a',
   'cuz2ili',
   'cuz3zwf',
   'cuz2kz8',
   'cuz3e8d',
   'cuz2y2h',
   'cuz3p8y',
   'cuz3mot',
   'cuzaunl',
   'cuztchh',
   'cuz2j6p'],
  'text': ["Hey, Network Engineer here! \n\n (please don't get mad at me mechanical/electrical/etc engineers, it's just a job title.)\n\nThat idea is actually the exact opposite of how the internet works. (funny enough Cisco, a major networking vendor, has a line of switches called Nexus.)\n\nStrictly speaking the internet routing is completely distributed.  The BGP protocol is how that routing table is updated and hundreds of thousands of people/companies update the table regularly.   Those updates include    the destination  information and ho

In [90]:
dataset['test_asks'][:5]


{'q_id': ['1xa1l8', '3ko13p', '2jy57p', '1943xl', '98va8l'],
 'title': ["If the heart is just a muscle, why can't I voluntarily flex it, or move it in any way?",
  'In the latest Avengers movie, there is a "Nexus" hub which routes every packet of traffic on the internet. What is the closest thing we have to a core of the internet in real life?',
  "Can somebody give me an intuitive understanding of Sine, Cosine, Tangent and perhaps Cotangent, Secant, Cosecant in such a way I won't forget?",
  "I've seen a magnet being dropped down a copper pipe... what happens if you do the opposite?",
  'What is the del operator in the Schrodinger equation?'],
 'selftext': ["Oftentimes, when when I focus on my heartbeat, I start to freak out that I might accidentally tell it to stop, or think something irrational like I could make it start beating manually, and then be stuck forever having to concentrate to contract my own heart to keep from dying. \n\nIf the heart is just a basic muscle, why can't I 

In [91]:
temp_ds = pd.DataFrame([])



In [92]:
temp_ds = dataset['test_asks'].to_pandas()
temp_ds

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,1xa1l8,"If the heart is just a muscle, why can't I vol...","Oftentimes, when when I focus on my heartbeat,...",,askscience,"{'a_id': ['cf9gyt1', 'cf9h410', 'cf9i6b2'], 't...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Cardiac...
1,3ko13p,"In the latest Avengers movie, there is a ""Nexu...",,,askscience,"{'a_id': ['cuz3k5a', 'cuz2ili', 'cuz3zwf', 'cu...",{'url': []},{'url': []},{'url': ['http://www.infoworld.com/article/261...
2,2jy57p,Can somebody give me an intuitive understandin...,I use them all the time for equations but they...,,askscience,"{'a_id': ['clg9vcq', 'clga65f', 'clga5ty', 'cl...",{'url': []},{'url': []},{'url': ['https://en.wikipedia.org/wiki/Unit_c...
3,1943xl,I've seen a magnet being dropped down a copper...,"If you had a magnetic tube, and dropped a piec...",,askscience,"{'a_id': ['c8knyk4', 'c8kphpb', 'c8klvyd', 'c8...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Moving_...
4,98va8l,What is the del operator in the Schrodinger eq...,,,askscience,"{'a_id': ['e4j02cf', 'e4jcydl', 'e4j8usk'], 't...",{'url': []},{'url': []},{'url': []}
...,...,...,...,...,...,...,...,...,...
4457,3fbqr1,Is it possible to accurately predict the resul...,This is probably a really basic chemistry ques...,,askscience,"{'a_id': ['ctoqy36', 'ctonyz7'], 'text': ['You...",{'url': []},{'url': []},{'url': ['https://en.wikipedia.org/wiki/Chemic...
4458,2t0vxd,Is the light clock model wrong?,"Learning special relativity, teachers like to ...",,askscience,"{'a_id': ['cnuqju4'], 'text': ['The clock mode...",{'url': []},{'url': ['http://galileoandeinstein.physics.vi...,{'url': []}
4459,w74y9,What animal does this belong to?,_URL_0_\n\nI found this spine the other day an...,,askscience,"{'a_id': ['c5auwkc', 'c5aua11'], 'text': ['The...",{'url': []},{'url': ['http://imgur.com/a/FRVMv#0']},{'url': []}
4460,9r6yq1,Is fluid mechanics and vector analysis applica...,Just curious if there is any overlap into biol...,,askscience,"{'a_id': ['e8ifhna'], 'text': ['Momentum and e...",{'url': []},{'url': []},{'url': []}


In [93]:
dataset['test_asks']

Dataset({
    features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
    num_rows: 4462
})

In [94]:
temp_ds = pd.concat([temp_ds, dataset['train_asks'].to_pandas()], ignore_index=True)
temp_ds

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,1xa1l8,"If the heart is just a muscle, why can't I vol...","Oftentimes, when when I focus on my heartbeat,...",,askscience,"{'a_id': ['cf9gyt1', 'cf9h410', 'cf9i6b2'], 't...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Cardiac...
1,3ko13p,"In the latest Avengers movie, there is a ""Nexu...",,,askscience,"{'a_id': ['cuz3k5a', 'cuz2ili', 'cuz3zwf', 'cu...",{'url': []},{'url': []},{'url': ['http://www.infoworld.com/article/261...
2,2jy57p,Can somebody give me an intuitive understandin...,I use them all the time for equations but they...,,askscience,"{'a_id': ['clg9vcq', 'clga65f', 'clga5ty', 'cl...",{'url': []},{'url': []},{'url': ['https://en.wikipedia.org/wiki/Unit_c...
3,1943xl,I've seen a magnet being dropped down a copper...,"If you had a magnetic tube, and dropped a piec...",,askscience,"{'a_id': ['c8knyk4', 'c8kphpb', 'c8klvyd', 'c8...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Moving_...
4,98va8l,What is the del operator in the Schrodinger eq...,,,askscience,"{'a_id': ['e4j02cf', 'e4jcydl', 'e4j8usk'], 't...",{'url': []},{'url': []},{'url': []}
...,...,...,...,...,...,...,...,...,...
136235,c3yf3z,"How much carbon dioxide is stored in leaves, t...",,,askscience,"{'a_id': ['erubgvz', 'ervj703', 'eru6zhw'], 't...",{'url': []},{'url': []},{'url': ['https://cdiac.ess-dive.lbl.gov/pns/c...
136236,q7db5,Why does cyanide kill you so quickly?,Hitler and many other German officials committ...,,askscience,"{'a_id': ['c3vc32o', 'c3vgpwl', 'c3vbhte', 'c3...",{'url': []},{'url': []},{'url': ['http://en.wikipedia.org/wiki/Death_o...
136237,163s6a,How does cyanide kill you?,I've been researching for a school paper but c...,,askscience,"{'a_id': ['c7sgpd7', 'c7sginx'], 'text': ['Cya...",{'url': []},{'url': []},{'url': ['http://www.jbc.org/content/269/39/24...
136238,itwn2,Why do LCD monitors have fixed refresh rates?,Why wouldn't an lcd monitor just be able to dr...,,askscience,"{'a_id': ['c26mrg1', 'c26nnpw'], 'text': ['Thi...",{'url': []},{'url': []},{'url': []}


In [95]:
df = pd.DataFrame([])

for i in list (dataset.keys() ):
  df = pd.concat([df, dataset[i].to_pandas()], ignore_index=True)

df

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []}
1,2lojul,Why are different tiers (regular < mid < premi...,I've noticed that the difference in price betw...,,explainlikeimfive,"{'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'], 't...",{'url': []},{'url': []},{'url': []}
2,8v5e3s,Stars and Visibility,Why do stars in the night's sky seem to disapp...,,explainlikeimfive,"{'a_id': ['e1kpw6u'], 'text': ['It's a quirk o...",{'url': []},{'url': []},{'url': []}
3,1v3wij,How do we know all the money the government is...,We hear about these large billion dollar bank ...,,explainlikeimfive,"{'a_id': ['ceohakd', 'ceoikhs', 'ceoji15', 'ce...",{'url': []},{'url': ['http://dealbook.nytimes.com/2014/01/...,{'url': ['http://www.reuters.com/article/2013/...
4,2jlp6f,What are good and bad sides of manual and auto...,Please consider I'm not a driver. Automatic se...,,explainlikeimfive,"{'a_id': ['clcur3j'], 'text': ['Automatics wei...",{'url': []},{'url': []},{'url': []}
...,...,...,...,...,...,...,...,...,...
558664,1cwijw,What was the biggest man-made loss of life in ...,,,AskHistorians,"{'a_id': ['c9kne6c'], 'text': ['The largest I ...",{'url': []},{'url': []},{'url': []}
558665,8fucgx,"Hi, I was hoping that someone would be able to...",,,AskHistorians,"{'a_id': ['dy6lps1'], 'text': ['In Western Eur...",{'url': []},{'url': []},{'url': []}
558666,u4oom,What was the US Government's reaction to the D...,Obviously now there is an infrastructure in pl...,,AskHistorians,"{'a_id': ['c4sasnw'], 'text': ['The New Dealer...",{'url': []},{'url': []},{'url': []}
558667,bh9auu,What did America do after the dust bowl,"Today I was learning about the dust bowl, whic...",,AskHistorians,"{'a_id': ['elrx4pm'], 'text': ['Agricultural p...",{'url': []},{'url': []},{'url': []}


In [96]:

# Lets check if the length of the dataframe is equal to the sum of the length of the individual datasets
len(df) == sum( [len(dataset[i]) for i in list (dataset.keys())] )

True

In [97]:
df['answers'][0]

{'a_id': array(['ccwtgnz', 'ccwtmho', 'ccwt946', 'ccwvj0u'], dtype=object),
 'text': array(["Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage.",
        "If you throw the ball all the time, then the defense will adapt to always cover for a pass.  By doing a simple running play every now and then, you force the defense to stay close and guard against the run.  Sometimes, the offense can catch the defense off guard by faking a run and freeing up their receivers.\n\nAlso, you don't have to gain massive yards on every single play.  Sometimes, it works best to gain a few yards at a time.  As long as you get the first down, you are in good shape.",
        'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays to the outside/throws the defense will catch on.\n\nAlso, 2 5 ya

In [98]:
df['answers'][0].get ('text', None)

array(["Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage.",
       "If you throw the ball all the time, then the defense will adapt to always cover for a pass.  By doing a simple running play every now and then, you force the defense to stay close and guard against the run.  Sometimes, the offense can catch the defense off guard by faking a run and freeing up their receivers.\n\nAlso, you don't have to gain massive yards on every single play.  Sometimes, it works best to gain a few yards at a time.  As long as you get the first down, you are in good shape.",
       'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays to the outside/throws the defense will catch on.\n\nAlso, 2 5 yard plays gets you a new set of downs.',
       "I you don't like those type of plays, w

In [99]:
import pandas as pd

# Sample DataFrame
data = {
    'answers': [
        {'text': 'Value 1', 'key2': 'Another Value'},
        {'text': 'Value 2', 'key2': 'More Value'},
        # Add more rows with dictionaries
    ]
}

df1 = pd.DataFrame(data)

# Extract 'text' values from 'answers' and create a new column 'new_column'
df1['new_column'] = df['answers'].apply(lambda x: x.get('text', None))

# Display the updated DataFrame
df1


,answers,new_column
0,"{'text': 'Value 1', 'key2': 'Another Value'}","[Keep the defense honest, get a feel for the p..."
1,"{'text': 'Value 2', 'key2': 'More Value'}","[As someone who uses quality Premium, I wish t..."


In [100]:
df['text_ans'] = df['answers'].apply(lambda x: x.get('text', None))
df

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,text_ans
0,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"[Keep the defense honest, get a feel for the p..."
1,2lojul,Why are different tiers (regular < mid < premi...,I've noticed that the difference in price betw...,,explainlikeimfive,"{'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'], 't...",{'url': []},{'url': []},{'url': []},"[As someone who uses quality Premium, I wish t..."
2,8v5e3s,Stars and Visibility,Why do stars in the night's sky seem to disapp...,,explainlikeimfive,"{'a_id': ['e1kpw6u'], 'text': ['It's a quirk o...",{'url': []},{'url': []},{'url': []},[It's a quirk of the human eye. At the center ...
3,1v3wij,How do we know all the money the government is...,We hear about these large billion dollar bank ...,,explainlikeimfive,"{'a_id': ['ceohakd', 'ceoikhs', 'ceoji15', 'ce...",{'url': []},{'url': ['http://dealbook.nytimes.com/2014/01/...,{'url': ['http://www.reuters.com/article/2013/...,[I'm pretty confident most of it isn't going b...
4,2jlp6f,What are good and bad sides of manual and auto...,Please consider I'm not a driver. Automatic se...,,explainlikeimfive,"{'a_id': ['clcur3j'], 'text': ['Automatics wei...",{'url': []},{'url': []},{'url': []},"[Automatics weigh more, so that alone makes ga..."
...,...,...,...,...,...,...,...,...,...,...
558664,1cwijw,What was the biggest man-made loss of life in ...,,,AskHistorians,"{'a_id': ['c9kne6c'], 'text': ['The largest I ...",{'url': []},{'url': []},{'url': []},[The largest I can think of is the Firebombing...
558665,8fucgx,"Hi, I was hoping that someone would be able to...",,,AskHistorians,"{'a_id': ['dy6lps1'], 'text': ['In Western Eur...",{'url': []},{'url': []},{'url': []},[In Western Europe there are very few specific...
558666,u4oom,What was the US Government's reaction to the D...,Obviously now there is an infrastructure in pl...,,AskHistorians,"{'a_id': ['c4sasnw'], 'text': ['The New Dealer...",{'url': []},{'url': []},{'url': []},[The New Dealers generally to a regional appro...
558667,bh9auu,What did America do after the dust bowl,"Today I was learning about the dust bowl, whic...",,AskHistorians,"{'a_id': ['elrx4pm'], 'text': ['Agricultural p...",{'url': []},{'url': []},{'url': []},[Agricultural practices changed in part by nec...


In [101]:
df['text_ans'][0]

array(["Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage.",
       "If you throw the ball all the time, then the defense will adapt to always cover for a pass.  By doing a simple running play every now and then, you force the defense to stay close and guard against the run.  Sometimes, the offense can catch the defense off guard by faking a run and freeing up their receivers.\n\nAlso, you don't have to gain massive yards on every single play.  Sometimes, it works best to gain a few yards at a time.  As long as you get the first down, you are in good shape.",
       'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays to the outside/throws the defense will catch on.\n\nAlso, 2 5 yard plays gets you a new set of downs.',
       "I you don't like those type of plays, w

In [102]:
df = df.explode('text_ans', ignore_index=True)

In [103]:
df['text_ans'][0]

"Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage."

In [104]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,text_ans
0,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"Keep the defense honest, get a feel for the pa..."
1,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"If you throw the ball all the time, then the d..."
2,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},In most cases the O-Line is supposed to make a...
3,1oy5tc,in football whats the point of wasting the fir...,,,explainlikeimfive,"{'a_id': ['ccwtgnz', 'ccwtmho', 'ccwt946', 'cc...",{'url': []},{'url': []},{'url': []},"I you don't like those type of plays, watch CF..."
4,2lojul,Why are different tiers (regular < mid < premi...,I've noticed that the difference in price betw...,,explainlikeimfive,"{'a_id': ['clwqgxl', 'clwqpjq', 'clwuh3s'], 't...",{'url': []},{'url': []},{'url': []},"As someone who uses quality Premium, I wish th..."


In [105]:
df['text_ans']

0          Keep the defense honest, get a feel for the pa...
1          If you throw the ball all the time, then the d...
2          In most cases the O-Line is supposed to make a...
3          I you don't like those type of plays, watch CF...
4          As someone who uses quality Premium, I wish th...
                                 ...                        
1442899    The largest I can think of is the Firebombing ...
1442900    In Western Europe there are very few specific ...
1442901    The New Dealers generally to a regional approa...
1442902    Agricultural practices changed in part by nece...
1442903    Timothy Mitchell's book *Rule of Experts* has ...
Name: text_ans, Length: 1442904, dtype: object

In [106]:
df['text_ans'][0]

"Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage."

In [107]:
df['text_ans'].apply

<bound method Series.apply of 0          Keep the defense honest, get a feel for the pa...
1          If you throw the ball all the time, then the d...
2          In most cases the O-Line is supposed to make a...
3          I you don't like those type of plays, watch CF...
4          As someone who uses quality Premium, I wish th...
                                 ...                        
1442899    The largest I can think of is the Firebombing ...
1442900    In Western Europe there are very few specific ...
1442901    The New Dealers generally to a regional approa...
1442902    Agricultural practices changed in part by nece...
1442903    Timothy Mitchell's book *Rule of Experts* has ...
Name: text_ans, Length: 1442904, dtype: object>